<a href="https://colab.research.google.com/github/1205apd/titanic/blob/main/IRIS_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
column_names=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
species=['Setosa','Versicolor','Virginica',]

In [4]:
df_train_path=tf.keras.utils.get_file('iris_train.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
df_test_path=tf.keras.utils.get_file('iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [5]:
df_train=pd.read_csv(df_train_path,names=column_names,header=0)
df_test=pd.read_csv(df_test_path,names=column_names,header=0)

In [6]:
df_train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [7]:
df_test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [8]:
y_train = df_train.pop('Species')
y_test = df_test.pop('Species')

In [9]:
df_train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [13]:
df_train.shape

(120, 4)

In [15]:
def input_fn(features,labels,training=True,batch_size=256):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset=dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [16]:
my_feature_columns=[]
for key in df_train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [18]:
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9palgx0w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## **TRAINING**

In [19]:
classifier.train(input_fn=lambda: input_fn(df_train,y_train,training=True),steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9palgx0w/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.6436496, step = 0
INFO:tensorflow:global_step/sec: 399.378
INFO:tensorflow:loss = 0.8394395, step = 100 (0.255 sec)
INFO:tensorflow:global_step/sec: 467.107
INFO:tensorflow:loss = 0.77433664, step = 200 (0.215 sec)
INFO:tensorflow:global_step/se

In [23]:
eval_result=classifier.evaluate(input_fn=lambda :input_fn(df_test,y_test,training=False))
print("\n Accuracy is: {accuracy:0.3f}\n".format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-26T13:39:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9palgx0w/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.24270s
INFO:tensorflow:Finished evaluation at 2021-06-26-13:39:19
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9, average_loss = 0.46202722, global_step = 5000, loss = 0.46202722
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp9palgx0w/model.ckpt-5000

 Accuracy is: 0.900



# PREDICTIONS

In [24]:
def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [32]:
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}
print('Please type Numeric Value as prompted')
for feature in features:
  valid=True
  while valid:
    val=input(feature+": ")
    if not val.isdigit(): valid = False
  predict[feature]=[float(val)]
predictions=classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]
  print('Prediction is "{}" ({:.1f}%)'.format(species[class_id],100*probability))

Please type Numeric Value as prompted
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9palgx0w/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (92.1%)
